In [1]:
import os
from langchain.document_loaders import TextLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import FAISS
from langchain.embeddings import HuggingFaceEmbeddings

In [ ]:
import os
from langchain.document_loaders import TextLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import FAISS
from langchain.embeddings import HuggingFaceEmbeddings


docs = []
splitter = RecursiveCharacterTextSplitter(chunk_size=200, chunk_overlap=50)  #decresed chunk size found this is the best for current index

for root, _, files in os.walk("ProcessGuides"):   #will go thru all sub folders in process guides folder
    for file in files:
        if file.endswith(".txt"):
            path = os.path.join(root, file)
            try:
                loader = TextLoader(path)
                text_docs = loader.load()
                
                # Access the text content from the Document object
                raw_content = text_docs[0].page_content if len(text_docs) > 0 else ""
                
                # Print the first 500 characters of the loaded content for debugging
                print(f"Raw content from {file}: {raw_content[:500]}")  # Show first 500 characters
                
                # Split the document into chunks
                chunks = splitter.split_documents(text_docs)
                
                # Print how many chunks were generated for this document
                print(f"Loaded {len(chunks)} chunks from {file}")
                
                # Add the chunks to the docs list
                docs.extend(chunks)
            except Exception as e:
                print(f"Failed to process {path}: {e}")

# Check if any documents were loaded
print(f"Total chunks loaded: {len(docs)}")
if len(docs) == 0:
    print("Warning: No documents were loaded. Please check the text files.")

#Load embedding model
embedding_model = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")

#Check the document sample before creating FAISS index
if len(docs) > 0:
    print(f"Sample document: {docs[0]}")

#Create and save FAISS index
try:
    vectorstore = FAISS.from_documents(docs, embedding_model)
    vectorstore.save_local("faiss_index")
    print(f"Embedded and saved {len(docs)} chunks to 'faiss_index/'")
except Exception as e:
    print(f"Error during FAISS creation: {e}")

Raw content from return_window.txt: 2 Days
    Return period of 2 days: Major Appliances* (e.g., Full Size Refrigerators, Freezers, Dishwashers, Oven Ranges, Cooktops, Furnaces, Trash Compactors, Wall Ovens, Washing Machine & Dryer Sets, Range Hoods, Washing Machines, Dryers, Ice makers) 



14 Days
    Marketplace Collectibles or Luxury Items*
    Most Wireless Phones


30 Days
    AppleCare+*
    Cooling & Heating (e.g., Air conditioners, Heaters, Air purifiers, Humidifiers, De-humidifiers, Fans, Indoor fireplaces)
    Consumer 
Loaded 7 chunks from return_window.txt
Raw content from restricted_returns.txt: Your item may have return restrictions and/or special instructions as follows:
    Wireless phones, Marketplace collectible and Marketplace luxury items (most are returnable within 14 days).
    
    
    Marketplace and Consumer Electronics items (most items are returnable within 30 days).
    
    
    Items that are part of Consumer Electronics or Collectible bundles must all b

/tmp/ipykernel_10769/512371970.py:43: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embedding_model = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
/home/lenue/Desktop/retail_bot_v1/retail_bot/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Sample document: page_content='2 Days' metadata={'source': 'ProcessGuides/returns/return_window.txt'}
Embedded and saved 36 chunks to 'faiss_index/'
